In [17]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

<ipython-input-17-0c95b2fc7292>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [16]:
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file, 
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

ValueError: time data '10/03/2004;18.00.00;2' does not match format '%Y-%m-%d %H:%M:%S'

In [ ]:
print(df)

In [3]:
# Visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn; seaborn.set()  # set plot styles
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

Using matplotlib backend: Qt5Agg


In [4]:
co = df['CO(GT)'].copy()
co.interpolate(inplace = True)

NameError: name 'df' is not defined

In [7]:
# Visualize original and imputed date
plt.plot(df['CO(GT)'], label = 'original', zorder = 2)
plt.plot(co, label = 'linear interpolation', zorder = 1)
plt.legend(loc= 'best')
plt.show()

NameError: name 'df' is not defined

In [8]:
# Detecting outliers using Boxplot
plt.boxplot(co)
plt.title('Detecting outliers using boxplot')
plt.xlabel('CO(GT)')
plt.show()

NameError: name 'co' is not defined

In [9]:
# Calculate correlations between variables 
corr_matrix = df.corr()
print(corr_matrix)

NameError: name 'df' is not defined

In [10]:
# choose the least corrlated variable 
rh = df['RH'].copy().interpolate() # 결측치까지 메꾸고 가져옴. (RH(Relative Humidity))
rh.plot()

NameError: name 'df' is not defined

In [11]:
# Visulize a scatter plot(CO, RH)
plt.scatter(co, rh, s = 8, c = 'black')
plt.xlabel('CO(GT)')
plt.ylabel('RH')
plt.show()

NameError: name 'co' is not defined

In [ ]:
# Choose the most correlated variable
nmhc = df['PT08.S2(NMHC)'].copy().interpolate() # NMHC: NON-methane HydroCarbon

In [ ]:
# Visulize a scater plot(CO, NMHC)
plt.scatter(co, nmhc, s = 12, c = 'black')
plt.xlabel('CO(GT)')
plt.ylabel('PT08.S2(NMHC)')
plt.show()

In [ ]:
"""
IQR-based Outlier Detection
"""

#Q1, Q2(median), Q3
q1 = co.quantile(0.25) # 25% value
median = co.quantile(0.5) # median
q3 = co.quantile(0.75) # 75% value
print(q1, median, q3)

In [ ]:
# IQR, upper_fence, lower_fence
iqr = q3 - q1
upper_fence = q3 + 1.5 * iqr
lower_fence = q1 - 1.5 * iqr
print(iqr, upper_fence, lower_fence) # lower fence는 음수 값이 나올 수 없기 때문에 0 임.

In [ ]:
# Filtering the outliers

outliers = co.loc[(co > upper_fence) | (co < lower_fence)] # 값이 모두 0 이상이기 때문에, lower fence보다 작을 순 없다.
print(outliers)

In [ ]:
# Mask for outliers
mask = co.index.isin(outliers.index)

In [ ]:
# Visulize the normal date and outliers
plt.plot(co[~mask], label = 'normal', color = 'blue',
        marker = 'o', markersize = 3, linestyle = 'None')
plt.plot(outliers, label = 'outliers', color = 'red',
        marker = 'x', markersize = 3, linestyle = 'None')
plt.legend(loc = 'best')
plt.show()

In [ ]:
# Removing the ouliers
co_refined = co.copy()
co_refined[mask] = np.nan
# co_refined.plot()

In [ ]:
# linear interpolation
co_refined.interpolate(inplace = True)
co_refined.plot()

In [ ]:
"""
Detecting Outliers with Z - scores
"""
#Visualize the distribution of the 'CO(GT)' variable
import seaborn as sns
sns.distplot(co)# distribution plot

In [ ]:
# Mean, Standard deviation
mean = np.mean(co)
std = np.std(co)
print(mean, std)

In [ ]:
# calculate Z - scores for each date points
outliers = []
thres = 3 # Z - score threshold

for i in co:
    z_score = (i-mean) / std
    if (np.abs(z_score) > thres):
        print(z_score)
        outliers.append(i)

In [ ]:
# Simplified version of filtering outliers
outliers = co.loc[np.abs((co - mean) / std) > thres].copy()

In [ ]:
# Mask for outliers
mask = co.index.isin(outliers.index)
mask[:50]

In [ ]:
# Comparison of distributions before outlier removal
sns.distplot(co, axlabel = 'CO(GT)', label = 'original')
sns.distplot(co[~mask], label = 'outliers removed')
plt.legend(loc = 'best')
plt.show()
# [exer] Adjust thres

In [ ]:
# Flooring and Copping
floor = co.quantile(0.1)
cap = co.quantile(0.9)
# Visualize the result
co.loc[co < floor] = floor
co.loc[co > cap] = cap
co.plot()